In [ ]:
import pandas as pd
import pickle
from tensorflow.keras.models import load_model

def load_prediction_assets():
    """Loads the Keras model, encoders, and scaler."""
    try:
        model = load_model('Classifier_Training/models/model.h5')
        with open('Classifier_Training/pipelines/label_encoder.pkl', 'rb') as f:
            label_encoder_gender = pickle.load(f)
        with open('Classifier_Training/pipelines/one_hot_encoding.pkl', 'rb') as f:
            one_hot_encoder = pickle.load(f)
        with open('Classifier_Training/pipelines/scaler.pkl', 'rb') as f:
            scaler = pickle.load(f)
        return model, label_encoder_gender, one_hot_encoder, scaler
    except FileNotFoundError as e:
        print(f"Fatal Error: Could not load model or pipeline files: {e}. The app cannot proceed.")
        return None, None, None, None


keras_model, label_encoder_gender, one_hot_encoder, scaler = load_prediction_assets()

def predict_churn(model, label_encoder, one_hot_encoder, scaler, input_data):
    try:
        input_df = pd.DataFrame([input_data])
        # One-hot encode Geography
        geo_one = one_hot_encoder.transform(input_df[["Geography"]])
        if hasattr(geo_one, "toarray"):
            geo_one = geo_one.toarray()
        geo_df = pd.DataFrame(geo_one, columns=one_hot_encoder.get_feature_names_out(["Geography"]))
        
        # Encode Gender
        input_df["Gender"] = label_encoder.transform(input_df["Gender"])
        
        # Merge into final dataframe
        final_df = pd.concat([input_df.drop(["Geography"], axis=1), geo_df], axis=1)
        # ✅ Reorder columns to match training order
        if hasattr(scaler, "feature_names_in_"):
            final_df = final_df[scaler.feature_names_in_]
        # Scale features
        scaled_array = scaler.transform(final_df)
        scaled_df = pd.DataFrame(scaled_array, columns=final_df.columns)
        print("Final ordered columns:", final_df.columns.tolist())
        
        # --- Prediction ---
        prediction = model.predict(scaled_df)
        proba = float(prediction[0][0])
        result = "⚠️ The customer is **likely to exit** the bank." if proba > 0.5 else "✅ The customer is **likely to stay** with the bank."
        
        return proba, result
    except Exception as e:
        return None, f"Prediction failed: {e}"
    

input_data = {
    'CreditScore': 650, 'Age': 38, 'Tenure': 5, 'Balance': 75000.0,
    'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1,
    'Geography': 'France', 'Gender': 'Male', 'EstimatedSalary': 100000.0
}

print(predict_churn(model=keras_model,
                    label_encoder=label_encoder_gender,
                    one_hot_encoder=one_hot_encoder,
                    scaler=scaler,
                    input_data=input_data))


Final ordered columns: ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France', 'Geography_Germany', 'Geography_Spain']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
(0.08083425462245941, '✅ The customer is **likely to stay** with the bank.')
